In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movie_csv = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')

In [ ]:
tags_csv= pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')

In [ ]:
tags_csv.head()

In [ ]:
from collections import defaultdict
movieid = defaultdict(list)
for i in range(0,len(tags_csv)):
    movieid[tags_csv['movieId'].iloc[i]].append(str(tags_csv['tag'].iloc[i]))

In [ ]:
movieidslist = []
moviemap = defaultdict()
j = 0
for key,values in movieid.items():
    movieidslist.append((' ').join(values))
    moviemap[key] = j
    j =j+1
    

In [ ]:
moviemap
moviemapreverse = defaultdict(list)
for key,value in moviemap.items():
    moviemapreverse[value] = key

In [ ]:
vectorizer = TfidfVectorizer(max_features  = 3000)
vectors = vectorizer.fit_transform(movieidslist)

Extracting features using autoencoders

In [ ]:
from keras import models
from keras.models import Model

from keras.layers import Dense,Embedding,Input,Flatten,Dropout
from keras.optimizers import Adam

In [ ]:
def getmodel(latent_factors,features):
    model = models.Sequential()
    model.add(Dense(4096,activation = 'relu', input_shape=(24111,)))
    model.add(Dense(1024,activation = 'relu'))
    model.add(Dense(512,activation= 'relu'))
    model.add(Dense(128,activation = 'relu'))
    model.add(Dense(latent_factors,activation = 'relu'))
    model.add(Dense(128,activation = 'relu'))
    model.add(Dense(256,activation = 'relu'))
    model.add(Dense(1024,activation = 'relu'))
    model.add(Dense(4096,activation = 'relu'))
    model.add(Dense(features,activation = 'sigmoid'))
    return model
                    
def getmodel1(latent_factors, features):
    inp1 = Input(shape=(features,))
    #d1 = Dense(4096, activation='relu')(inp1)
    #d1 = Dropout(0.3)(d1)
    #d1 = Dense(1024, activation='relu')(inp1)
    #d1 = Dense(512, activation='relu')(inp1)
    #d1 = Dense(128, activation='relu')(d1)
    d1 = Dense(latent_factors, activation='relu')(inp1)
   #d1 = Dense(128, activation='relu')(d1)
    #d1 = Dense(512, activation='tanh')(d1)
    #d1 = Dense(1024, activation='relu')(d1)
    #|d1 = Dense(4096, activation='relu')(d1)
    d1 = Dense(features, activation='sigmoid')(d1)
    model = Model(inputs=inp1, outputs=d1)
    
    return model

In [ ]:
print(vectors.shape)
features = vectors.shape[1]
#print(features.shape)
#features = features.reshape(-1,)
latentfactors = 32
model = getmodel1(latentfactors,features)
model.compile(loss = 'binary_crossentropy',optimizer = Adam(0.01),metrics = ['acc'])
model.summary()

In [ ]:
model_autoencoder = model.fit(array,array,epochs = 1,validation_split = 0.1)

In [ ]:
model.save('/kaggle/working/model_autoencoder.h5')

In [ ]:
model.summary()

In [ ]:
modell = models.Sequential()
count = 0
for i in model.layers:
    if(count<2):
        modell.add(i)
    count = count+1

In [ ]:
modell.summary()

In [ ]:
def getinfo(x):
    iid = getid(x)
    moviename = np.where(movie_csv['movieId'] == iid)
    moviename = list(moviename)[0]
    moviename = moviename[0]
    print("movie_name :{}".format(movie_csv.iloc[moviename]))
    print('\n Movie descriptions: {}'.format(movieidslist[x]))


In [ ]:
a = []
a.append(movieidslist[875])
#getinfo(875)
vector_item = vectorizer.transform(a)
ans = modell.predict(vector_item)

In [ ]:
b = []
b.append(movieidslist[87])
#getinfo(87)
vector_items1 = vectorizer.transform(b)
ansa = modell.predict(vector_items1)

In [ ]:
val = cosine_similarity(ans,ansa)

In [ ]:
cosines = []
for i in predictlistvectors:
    sim = cosine_similarity(ans,i)
    cosines.append(sim)

In [ ]:
from collections import defaultdict
cosines_id = []
for i,j in enumerate(cosines):
    cosines_id.append([i,j])

In [ ]:
def keysort(cosine_id):
    return cosine_id[1]
    

In [ ]:
cosines_id.sort(reverse= True,key = keysort)

In [ ]:
for i in cosines_id[0:10]:
    getinfo(i[0])